In [1]:
import cv2
import time
import wave
import pyttsx3
import argparse
import numpy as np 

from matplotlib import pyplot as plt
from time import sleep
from pynq.overlays.base import BaseOverlay

base = BaseOverlay("base.bit")
pAudio = base.audio

In [2]:
def load_yolo():
	net = cv2.dnn.readNet("yolov3.weights", "yolov3.cfg")
	classes = []
	with open("coco.names", "r") as f:
		classes = [line.strip() for line in f.readlines()] 
	
	output_layers = [layer_name for layer_name in net.getUnconnectedOutLayersNames()]
	colors = np.random.uniform(0, 255, size=(len(classes), 3))
	return net, classes, colors, output_layers

def load_image(img_path):
	# image loading
	img = cv2.imread(img_path)
	img = cv2.resize(img, None, fx=0.4, fy=0.4)
	height, width, channels = img.shape
	return img, height, width, channels

def detect_objects(img, net, outputLayers):			
	blob = cv2.dnn.blobFromImage(img, scalefactor=0.00392, size=(320, 320), mean=(0, 0, 0), swapRB=True, crop=False)
	net.setInput(blob)
	outputs = net.forward(outputLayers)
	return blob, outputs

def get_box_dimensions(outputs, height, width):
	boxes = []
	confs = []
	class_ids = []
	for output in outputs:
		for detect in output:
			scores = detect[5:]
			#print(scores)
			class_id = np.argmax(scores)
			conf = scores[class_id]
			if conf > 0.3:
				center_x = int(detect[0] * width)
				center_y = int(detect[1] * height)
				w = int(detect[2] * width)
				h = int(detect[3] * height)
				x = int(center_x - w/2)
				y = int(center_y - h / 2)
				boxes.append([x, y, w, h])
				confs.append(float(conf))
				class_ids.append(class_id)
	return boxes, confs, class_ids

def draw_labels(boxes, confs, colors, class_ids, classes, img): 
	indexes = cv2.dnn.NMSBoxes(boxes, confs, 0.5, 0.4)
	font = cv2.FONT_HERSHEY_PLAIN
	for i in range(len(boxes)):
		if i in indexes:
			x, y, w, h = boxes[i]
			label = str(classes[class_ids[i]])
			print(label)
			color = colors[i]
			color1 = (255,0,0)
			cv2.rectangle(img, (x,y), (x+w, y+h), color, 2)
			cv2.putText(img, label, (x, y - 5), font, 4, color1, 2)
	#cv2.imshow("Image", img) #a hdmi monitor/display is required to run this function
	plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
	plt.show() 
	return label
    
def start_webcam():
	cap = cv2.VideoCapture(0)
	return cap
    
def webcam_detect():
	model, classes, colors, output_layers = load_yolo()
	cap = start_webcam()
	while True:
		_, frame = cap.read()
		height, width, channels = frame.shape
		blob, outputs = detect_objects(frame, model, output_layers)
		boxes, confs, class_ids = get_box_dimensions(outputs, height, width)
		label = draw_labels(boxes, confs, colors, class_ids, classes, frame)
		return label
		key = cv2.waitKey(1)
		if key == 27:
			break
	cap.release()

In [3]:
# Initialize the pyttsx3 engine
engine = pyttsx3.init()

# Set properties for the speech output (optional)
engine.setProperty('rate', 150)  # Speed of speech
engine.setProperty('volume', 1.0)  # Volume (0.0 to 1.0)

# Set the output file name
output_file = 'output.wav'

In [4]:
volume = 5
while True:
    global volume
    if (base.buttons[0].read()==1):
        volume = volume + 1
        if (volume > 10):
            volume = 10
        engine.setProperty('volume', volume/10)
        
    if (base.buttons[1].read()==1):
        volume = volume - 1
        if (volume < 0):
            volume = 0
        engine.setProperty('volume', volume/10)
        
    detected_label = webcam_detect()
    text = detected_label
    engine.save_to_file(text, output_file)

    # Run the speech synthesis
    engine.runAndWait()
    pAudio.load("output.wav")
    pAudio.play()